### 1. 데이터 로드 및 전처리

* 훈련데이터로 병렬 코퍼스 로드

In [1]:
import os
import re
import shutil
import zipfile

import numpy as np
import pandas as pd
import tensorflow as tf
import unicodedata
import urllib3
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [3]:
# fra-eng. zip 파일 로드 : 샘플 33000개만 추출
num_samples = 33000

# 전처리 함수 구현
def to_ascii(s):
    # 프랑스 악센트 삭제
    return ''.join(c for c in unicodedata.normalize('NFD', s) \
                   if unicodedata.category(c) != 'Mn')

def preprocess_sentence(sent):
    # 악센트 제거 함수 호출
    sent = to_ascii(sent.lower())
    
    # 단어와 구두점 사이에 공백 추가
    ## ex) "I am a student." => "I am a student ."
    sent = re.sub(r'([?.!,¿])', r' \1', sent)
    
    # (a-z, A-Z, ".", "?", "!", ",") 이들을 제외하고는 전부 공백으로 변환.
    sent = re.sub(r'[^a-zA-Z!.?]+', r' ', sent)
    
    # 여러 공백을 하나의 공백으로 치환
    sent = re.sub(r'\s+', ' ', sent)
    return sent

In [4]:
# 전처리 테스트
en_sent = u"Have you had dinner?"
fr_sent = u"Avez-vous déjà diné?"

print('전처리 전 영어 문장:', en_sent)
print('전처리 후 영어 문장:', preprocess_sentence(en_sent))

전처리 전 영어 문장: Have you had dinner?
전처리 후 영어 문장: have you had dinner ?


In [5]:
print('전처리 전 프랑스어 문장:', fr_sent)
print('전처리 후 프랑스어 문장:', preprocess_sentence(fr_sent))

전처리 전 프랑스어 문장: Avez-vous déjà diné?
전처리 후 프랑스어 문장: avez vous deja dine ?


In [13]:
# 샘플 데이터에 대해 전처리 수행
# 교사 강요를 사용할 예정이므로, 훈련시 사용할 디코더의 입력시퀀스와 실제값, 
# 즉, 레이블에 해당되는 출력 시퀀스를 따로 분리하여 지정
# 입력시퀀스에는 시작을 의미하는 토큰인 <sos>를 추가하고, 
# 출력 시퀀스에는 종료를 의미하는 토큰인 <eos>를 추가함

def load_preprocessed_data():
    encoder_input, decoder_input, decoder_target = [], [], []
    
    with open('fra.txt', 'r', encoding='utf8') as lines:
        for i, line in enumerate(lines):
            # source 데이터와 target 데이터 분리
            src_line, tar_line, _ = line.strip().split('\t')
            
            # source 데이터 전처리
            src_line = [w for w in preprocess_sentence(src_line).split()]
            
            # target 데이터 전처리
            tar_line = preprocess_sentence(tar_line)
            tar_line_in = [w for w in ("<sos> " + tar_line).split()]
            tar_line_out = [w for w in (tar_line + " <eos>").split()]
            
            encoder_input.append(src_line)
            decoder_input.append(tar_line_in)
            decoder_target.append(tar_line_out)
            
            if i == num_samples - 1:  # 33,000개만 추출
                break
                
    return encoder_input, decoder_input, decoder_target

In [14]:
sents_en_in, sents_fra_in, sents_fra_out = load_preprocessed_data()

In [15]:
print('인코더의 입력:', sents_en_in[:5])
print('디코더의 입력:', sents_fra_in[:5])
print('디코더의 출력:', sents_fra_out[:5])

인코더의 입력: [['go', '.'], ['go', '.'], ['go', '.'], ['go', '.'], ['hi', '.']]
디코더의 입력: [['<sos>', 'va', '!'], ['<sos>', 'marche', '.'], ['<sos>', 'en', 'route', '!'], ['<sos>', 'bouge', '!'], ['<sos>', 'salut', '!']]
디코더의 출력: [['va', '!', '<eos>'], ['marche', '.', '<eos>'], ['en', 'route', '!', '<eos>'], ['bouge', '!', '<eos>'], ['salut', '!', '<eos>']]


* 여기에서 디코더의 입력 데이터는 교사강요에 사용됨

---
###### 케라스 토크나이저를 통해 단어집합생성 --> 정수인코딩 --> 패딩 진행

In [23]:
def process_padding(data):
    tokenizer = Tokenizer(filters="", lower=False)
    tokenizer.fit_on_texts(data)
    sequenced_data = tokenizer.texts_to_sequences(data)
    padded = pad_sequences(sequenced_data, padding='post')
    return padded, tokenizer

encoder_input, tokenizer_en = process_padding(sents_en_in)
decoder_input, tokenizer_fra = process_padding(sents_fra_in)
decoder_target, _ = process_padding(sents_fra_out)

In [24]:
print('인코더의 입력크기: ',  encoder_input.shape)
print('디코더의 입력크기: ',  decoder_input.shape)
print('디코더의 레이블 크기: ',  decoder_target.shape)

인코더의 입력크기:  (33000, 8)
디코더의 입력크기:  (33000, 16)
디코더의 레이블 크기:  (33000, 16)


* 샘플은 총 33,000개이며, 영어 문장의 길이는 8, 프랑스어 문장 길이는 16임
* 단어 집합 크기를 정의함

In [25]:
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_fra.word_index) + 1
print(src_vocab_size, tar_vocab_size) # 단어집합의 크기

4620 8053


In [26]:
# 단어로부터 정수를 얻는 딕셔너리와 정수로부터 단어를 얻는 디셔너리 만듦
src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word
tar_to_index = tokenizer_fra.word_index
index_to_tar = tokenizer_fra.index_word

In [27]:
# 테스트 데이터를 분리하기 전 shuffle 필요
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print('랜덤 시퀀스 :', indices)

랜덤 시퀀스 : [  838 19925 28428 ... 26048 24876  7157]


In [28]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [29]:
# check!!
print(encoder_input[30997])
print(decoder_input[30997])
print(decoder_target[30997])

[54 83  1  0  0  0  0  0]
[  1 160  37  12   8  92   2   0   0   0   0   0   0   0   0   0]
[160  37  12   8  92   2   1   0   0   0   0   0   0   0   0   0]


In [31]:
# 테스트 데이터 분리 --> 10%
n_of_val = int(33000 / 10)
n_of_val

3300

In [32]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [33]:
print('훈련 source 데이터의 크기 :',encoder_input_train.shape)
print('훈련 target 데이터의 크기 :',decoder_input_train.shape)
print('훈련 target 레이블의 크기 :',decoder_target_train.shape)
print('테스트 source 데이터의 크기 :',encoder_input_test.shape)
print('테스트 target 데이터의 크기 :',decoder_input_test.shape)
print('테스트 target 레이블의 크기 :',decoder_target_test.shape)

훈련 source 데이터의 크기 : (29700, 8)
훈련 target 데이터의 크기 : (29700, 16)
훈련 target 레이블의 크기 : (29700, 16)
테스트 source 데이터의 크기 : (3300, 8)
테스트 target 데이터의 크기 : (3300, 16)
테스트 target 레이블의 크기 : (3300, 16)


### 2. 모델 설계 : 번역기 만들기

In [34]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model

In [36]:
# 임베딩 벡터의 차원과 LSTM 은닉상태의 크기를 64로 사용
embedding_dim = 64
hidden_units = 64

* 인코더 설계 : Masking은 패딩 토큰인 숫자 0의 경우, 연산을 제외하는 역할 수행, 인코더의 내부상태를 디코더로 넘겨주어야 하기 때문에 return_state=True로 설정
* LSTM : state_h, state_c를 리턴받고, 이 두가지 상태 모두를 디코더로 전달(컨텍스트 벡터)

In [37]:
# 인코더 설계
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(src_vocab_size, embedding_dim)(encoder_inputs) # 임베딩층
enc_masking = Masking(mask_value=0.0)(enc_emb) # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(hidden_units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)
encoder_states = [state_h, state_c]

* 디코더는 인코더의 마지막 은닉상태로부터 초기 은닉상태를 얻음
* inital_state의 인자값으로 encoder_states를 주는 코드가 이에 해당함
* 디코더도 은닉상태, 셀상태를 리턴하기는 하지만, 훈련과정에서는 사용하지 않음
* seq2seq의 디코더는 기본적으로 각 시점마다 다중 클래스 분류문제를 풀고 있음
* 따라서 출력층으로 소프트맥스 함수와 손실함수를 크로스 엔트로피 함수를 사용함

---
* categorical_crossentropy를 사용하려면 레이블은 원-핫 인코딩이 된 상태여야 함
* 그러나 원-핫 인코딩을 하지 않은 상태로 정수 레이블에 대해 다중 클래스 분류 문제를 풀고자 하는 경우에는 'categorical_crossentropy'를 사용할 수 있음

In [38]:
# 디코더
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(tar_vocab_size, embedding_dim) # embedding layer
dec_emb = dec_emb_layer(decoder_inputs)
dec_masking = Masking(mask_value=0.0)(dec_emb)

# 상태값 리턴을 위해 return_state=True, 
# 모든 시점에 대해 단어를 예측하기 위해 return_sequences=True
decoder_lstm = LSTM(hidden_units, return_state=True, return_sequences=True)

# 인코더의 은닉상태를 초기 은닉상태(initial state)로 사용
decoder_outputs, _, _ = decoder_lstm(dec_masking, initial_state=encoder_states)

# 모든 시점의 결과에 대해 소프트맥스 함수를 사용한 출력층을 통해 단어 예측
decoder_dense = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# 모델의 입력과 출력 정의
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model. compile(optimizer='adam', loss='sparse_categorical_crossentropy', \
               metrics=['acc'])

In [40]:
# training : 120개 배치크기, 50 에포크 학습
29700 / 120  # 247번 나눠서 진행하고 이 프로세스를 50번 수행

247.5

In [41]:
model.fit(x=[encoder_input_train, decoder_input_train], y=decoder_target_train, \
          validation_data=([encoder_input_test, decoder_input_test], decoder_target_test), \
          batch_size=120, epochs=50)

Epoch 1/50
248/248 [==============================] - 55s 223ms/step - loss: 3.2678 - acc: 0.6122 - val_loss: 1.9316 - val_acc: 0.6393
Epoch 2/50
248/248 [==============================] - 73s 293ms/step - loss: 1.7901 - acc: 0.7158 - val_loss: 1.6932 - val_acc: 0.7422
Epoch 3/50
248/248 [==============================] - 121s 486ms/step - loss: 1.6039 - acc: 0.7483 - val_loss: 1.5421 - val_acc: 0.7554
Epoch 4/50
248/248 [==============================] - 125s 502ms/step - loss: 1.4744 - acc: 0.7595 - val_loss: 1.4425 - val_acc: 0.7633
Epoch 5/50
248/248 [==============================] - 121s 489ms/step - loss: 1.3774 - acc: 0.7734 - val_loss: 1.3551 - val_acc: 0.7831
Epoch 6/50
248/248 [==============================] - 117s 471ms/step - loss: 1.2921 - acc: 0.7902 - val_loss: 1.2874 - val_acc: 0.7933
Epoch 7/50
248/248 [==============================] - 113s 456ms/step - loss: 1.2264 - acc: 0.7991 - val_loss: 1.2349 - val_acc: 0.8031
Epoch 8/50
248/248 [==============================

### 3. seq2seq 기계번역기 동작

In [42]:
# 인코더
encoder_model = Model(encoder_inputs, encoder_states)

# 디코더 설계
## 이전 시점의 상태를 저장할 텐서 wjddml
decoder_state_input_h = Input(shape=(hidden_units,))
decoder_state_input_c = Input(shape=(hidden_units,))
decoder_states_input = [decoder_state_input_h, decoder_state_input_c]

# 훈련때 사용했던 임베딩 층 재사용
dec_em2 = dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 이전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_em2, 
                                                    initial_state=decoder_states_input)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

# 수정된 디코더
decoder_model = Model(
    [decoder_inputs] + decoder_states_input, 
    [decoder_outputs2] + decoder_states2)

* 테스트 단계의 동작을 위한 decode_sequence 함수 구현
* 입력문장이 들어오면 인코더는 마지막 시점까지 전개하여 마지막 시점의 은닉상태와 셀 상태를 리턴함
* 두개의 값을 states_value에 저장하고, 디코더의 초기입력으로 \<sos>를 준비함
* 이를 target_seq에 저장함
* 이 두가지 입력을 가지고 while문에서 디코더 입력으로 사용
* 이러한 디코더는 현재 시점에 대해 예측하게 되며, 현재시점의 예측벡터는 output_tokens, 현재 시점의 은닉상태가 h, 현재시점의 셀 상태가 c임
* 예측 벡터로부터 현재시점의 예측 단어인 target_seq를 얻고, h와 c 두개의 값은 states_value에 저장함
* while 문의 다음 루프, 즉 두번째 시점의 디코더 입력으로 다시 target_seq와 states_value를 사용함
* 이를 현재 시점의 예측단어로 \<eos>를 예측하거나 번역 문장의 길이가 50이 넘는 순간까지 반복함
* 각 시점마다 번역된 단어는 decoded_sentence에 누적되면서 저장되었다가 최종 번역 시퀀스로 리턴됨

In [74]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 마지막 시점의 상태(은닉 상태, 셀 상태)를 얻음
    states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 정수 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = tar_to_index['<sos>']

    stop_condition = False
    decoded_sentence = ''

  # stop_condition이 True가 될 때까지 루프 반복
  # 구현의 간소화를 위해서 이 함수는 배치 크기를 1로 가정합니다.
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 단어로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

        # 현재 시점의 예측 단어를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 정해진 길이를 넘으면 중단.
        if (sampled_char == '<eos>' or len(decoded_sentence) > 50):
            stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

* 결과 확인을 위한 함수 만들기
* seq_to_src 함수는 영어 문장에 해당하는 정수 시퀀스를 입력받으면 정수로부터 영어단어를 리턴하는 index_to_src를 통해 영어문장으로 변환함
* seq_to_tar는 프랑스어에 해당하는 정수 시퀀스를 입력받으면 정수로부터 프랑스어 단어를 리턴하는 index_to_tar를 통해 프랑스어 문장으로 변환함

In [86]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_src(input_seq):
    sentence = ''
    for encoded_word in input_seq:
        if(encoded_word != 0):
            sentence = sentence + index_to_src[encoded_word] + ' '
    return sentence

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_tar(input_seq):
    sentence = ''
    for encoded_word in input_seq:
        if(encoded_word != 0):
            sentence = sentence + index_to_tar[encoded_word] + ' '
    return sentence

* 훈련 데이터 중 임의의 샘플 출력

In [88]:
# seq_to_tar 함수 not working well!!! ===> 수정 필요
for seq_index in [3, 50, 100, 300, 1001]:
    input_seq = encoder_input_train[seq_index: seq_index+1]
    decoded_sentence = decode_sequence(input_seq)
    
    print("입력문장: ", seq_to_src(encoder_input_train[seq_index]))
    print('정답문장: ', seq_to_tar(decoder_input_train[seq_index]))
    print('번역문장: ', decoded_sentence)
    print('-' * 50)

입력문장:  can i borrow yours ? 
정답문장:  <sos> puis je emprunter le tien ? 
번역문장:   puis je emprunter le tien ? <sos> <sos> <sos> <sos>
--------------------------------------------------
입력문장:  i ran out of gas . 
정답문장:  <sos> je suis tombee en panne d essence . 
번역문장:   je suis tombee a l interieur . <sos> <sos> <sos> <sos>
--------------------------------------------------
입력문장:  i really need this . 
정답문장:  <sos> j en ai vraiment besoin . 
번역문장:   j ai besoin d une biere . <sos> <sos> <sos> <sos> ca
--------------------------------------------------
입력문장:  water the plants . 
정답문장:  <sos> arrosez les plantes ! 
번역문장:   arrosez les plantes ! <sos> <sos> <sos> <sos> <sos>
--------------------------------------------------
입력문장:  go to bed . 
정답문장:  <sos> allez au lit ! 
번역문장:   va au lit ! <sos> <sos> <sos> <sos> <sos> <sos> <sos>
--------------------------------------------------


In [ ]:
decpder